In [1]:
#r "nuget: Oryx"
#r "nuget: Oryx.ThothJsonNet"

open Thoth.Json.Net

open Oryx
open Oryx.ThothJsonNet.ResponseReader

open System.Net.Http

type QuestradeAuthResponse = 
    { AccessToken: string
      ApiServer: string
      RefreshToken: string }

[<Literal>]
let Url = "https://login.questrade.com/oauth2/token"

let query refreshToken = [
    struct ("grant_type", "refresh_token")
    struct ("refresh_token", refreshToken)
]

let request refreshToken =
    GET
    >=> withUrl Url
    >=> withQuery (query refreshToken)
    >=> fetch
    >=> json<QuestradeAuthResponse> (Decode.Auto.generateDecoder<QuestradeAuthResponse>(caseStrategy=SnakeCase))

let httpClient = new HttpClient()

let context = 
    HttpContext.defaultContext
    |> HttpContext.withHttpClient httpClient

let resultTask =
    request "XXXXXXXXXXXXXXXXXXXX"
    |> runAsync context

let authResponse =
    resultTask
    |> Async.AwaitTask
    |> fun a -> async { let! result = a
    match result with
    | Ok x -> return $"OK: {x}"
    | Error e -> return $"Oops: {string <| e.ToDisplayString()}"  }
    |> Async.RunSynchronously

authResponse

Oops: { Newtonsoft.Json.JsonReaderException: Unexpected character encountered while parsing value: B. Path '', line 0, position 0.
   at Newtonsoft.Json.JsonTextReader.ParseValueAsync(CancellationToken cancellationToken)
   at Newtonsoft.Json.Linq.JToken.ReadFromAsync(JsonReader reader, JsonLoadSettings settings, CancellationToken cancellationToken)
   at Oryx.ThothJsonNet.Decode.body@1-3[T](FSharpFunc`2 decoder, JsonTextReader _arg2)
   at Oryx.ThothJsonNet.Decode.decodeStreamAsync@18-3.Invoke(Unit unitVar0)
   at Ply.TplPrimitives.tryFinally[u](FSharpFunc`2 continuation, FSharpFunc`2 finallyBody)
   at Oryx.ThothJsonNet.Decode.decodeStreamAsync@16-1.Invoke(Unit unitVar0)
   at Ply.TplPrimitives.tryFinally[u](FSharpFunc`2 continuation, FSharpFunc`2 finallyBody)
   at Ply.TplPrimitives.ContinuationStateMachine`1.System-Runtime-CompilerServices-IAsyncStateMachine-MoveNext()
   at Oryx.ThothJsonNet.ResponseReader.json@24-1.Invoke(Unit unitVar)
   at Ply.TplPrimitives.ContinuationStateMac